In [5]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [6]:
from crewai import Crew, Agent, Task

international_chef = Agent(
    role="International Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home",
    backstory="""
    You are an famous chef that specializes in couisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    verbose=True,
    allow_delegation=False,
)

healthy_chef = Agent(
    role="Healthy Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory="""
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    allow_delegation=False,
)

In [7]:
normal_recipe = Task(
    description="Come up with a {dish} that serves {people} people.",
    agent=international_chef,
    expected_output="Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions",
    output_file="normal_recipe.md",
)

healthy_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=healthy_chef,
    expected_output="Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.",
    output_file="healthy_recipe.md",
)

In [8]:
crew = Crew(
    tasks=[
        normal_recipe,
        healthy_recipe,
    ],
    agents=[
        international_chef,
        healthy_chef,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        dish="Korea dinner",
        people="5",
    )
)

2024-07-22 15:36:47,481 - 8380255232 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: International Chef
 [INFO]: == Starting Task: Come up with a Korea dinner that serves 5 people.


> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer:  

### Ingredients Required (Serves 5):  

#### For the Korean BBQ Beef (Bulgogi):  
- 1.5 lbs (680g) beef sirloin, thinly sliced  
- 1/2 cup soy sauce  
- 1/4 cup brown sugar  
- 1/4 cup sesame oil  
- 4 cloves garlic, minced  
- 1 inch ginger, grated  
- 1/4 cup green onions, chopped  
- 1 tablespoon sesame seeds  
- 1 tablespoon gochujang (Korean chili paste) - optional for heat  
- 1 tablespoon rice vinegar  
- 1/4 teaspoon black pepper  

#### For the Stir-Fried Vegetables:  
- 2 cups napa cabbage, chopped  
- 1 cup carrots, julienned  
- 1 cup bell peppers (red and green), sliced  
- 1 cup mushrooms, sliced  
- 2 tablespoons vegetable oil  
- 2 cloves garlic, minced  
- 1 tablespoon soy sauce  
- 1 tablespoon sesame oil  

#### For the Rice:  
- 2 cups jasmine rice  
- 